In [1]:
# from sst and rainfall ---> rainfall
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt 
import tensorflow.compat.v1 as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv3D, Conv2D
from keras.layers import ConvLSTM2D
from keras.layers import Attention
from keras.layers import Input
from keras.models import Model
from keras.layers import Dropout, Lambda
from keras import optimizers
from keras.layers import Conv2DTranspose, TimeDistributed
from keras import callbacks
import xarray as xr

# Define global variables
lead_time = 3
save_path = "/home/cccr/roxy/matin/MTech_project/model/Conv-LSTM/7in4out/"
model_path = save_path + "ConvLstm_sst13rf_rf_3SI.h5"
add_data = "/home/cccr/roxy/matin/MTech_project/data/"

def preprocess_data(sequence, n_steps, channels):
    """
    Function to preprocess the data and prepare it for model training.
    
    Parameters:
        sequence (xarray.Dataset): Data sequence to be preprocessed.
        n_steps (int): Number of time steps in the input sequence.
        channels (int): Number of channels in the input sequence.
    
    Returns:
        tuple: A tuple containing the preprocessed input sequence (X) and output sequence (y).
    """
    # Select data for Jun, Jul, Aug, Sept
    sequence = sequence.sel(time=slice("1998-01-01","2016-12-31"))
    sequence = sequence.where(sequence.time.dt.month.isin([6,7,8,9]), drop=True)
    max = sequence.max()
    min = sequence.min()
    
    # Max-min normalization
    sequence = (sequence-min)/(max - min)
    sequence = np.array(sequence)
    
    # Exponential Space Transform
    valid_pts = np.where(sequence==sequence)
    sequence[valid_pts] = np.exp(sequence[valid_pts])
    sequence[valid_pts] = np.power(sequence[valid_pts],7)
    invalid_pts = np.where(sequence != sequence)
    sequence[invalid_pts] = 0
    
    # Prepare input and output sequences
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix + lead_time > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix + lead_time]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


def create_input(channels, n_steps):
    """
    Function to create the input for the model.
    
    Parameters:
        channels (list): List of channels to be used in the input.
        n_steps (int): Number of time steps in the input sequence.
        
    Returns:
        numpy.ndarray: The input for the model.
    """
    stack = []
    for i in range(len(channels)):
        channel_data = xr.open_dataarray(f"/home/cccr/roxy/matin/MTech_project/data/{channels[i]}")
        input, _ = preprocess_data(channel_data, n_steps=n_steps, channels=channels[i])
        input = np.expand_dims(input, axis=2)
        print(f"Adding channel {i} with shape: {input.shape}")
        stack.append(input)
    out = np.dstack(stack)
    return out

channels = ["SIFilteredrfBOB_6lag.nc","SIFilteredSSTBOB_6.nc"]
steps = 7

# Create input for the model
input = create_input(channels, steps)

# Preprocess target data
channel_data = xr.open_dataarray(add_data + "SIFilteredrfBOB_6lag.nc")
_, target = preprocess_data(channel_data, n_steps=steps, channels=channels[-1])
target = np.expand_dims(target, axis=1)

# Roll axes for input and target
x = np.rollaxis(input, 4, 2)
x = np.rollaxis(x, 4, 2)

y = np.rollaxis(target, 3, 1)
y = np.rollaxis(y, 3, 1)

# Assert that input and target shapes are consistent
assert input.shape[0] == target.shape[0]
assert input.shape[-1] == target.shape[-1]
assert input.shape[-2] == target.shape[-2]


# Print shape of input and target
print("INPUT SHAPE --> ", input.shape)
print("TARGET SHAPE --> ", target.shape)

# Delete original input and target variables to free up memory
del input
del target

# Perform additional processing or modeling steps as needed
seq = tf.keras.Sequential()

# Add layers to the model
seq.add(ConvLSTM2D(filters=4, kernel_size=(3,3), padding='same', input_shape=(7,25,39,2), return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=8, kernel_size=(3,3), padding='same', return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=8, kernel_size=(3,3), padding='same', return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=16, kernel_size=(3,3), padding='same', return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=16, kernel_size=(3,3), padding='same', return_sequences=False, data_format='channels_last'))
seq.add(Conv2D(filters=15, kernel_size=(3,3), activation='relu', padding='same', data_format='channels_last'))
seq.add(Conv2D(filters=1, kernel_size=(3,3), activation='relu', padding='same', data_format='channels_last'))
# Create the query, value, and key inputs
query_input = Input(shape=(25,39,2), name='query_input')
value_input = Input(shape=(25,39,2), name='value_input')
key_input = Input(shape=(25,39,2), name='key_input')

# Pass the inputs to the Attention layer
attention_output = Attention(name='attention')([query_input, value_input, key_input])
Adam = tf.keras.optimizers.Adam(learning_rate=10**-4)
seq.compile(loss='mean_squared_error', optimizer=Adam, metrics=['mae'])

print(seq.summary())

# Define the number of training epochs
n_epochs = 200
logdir = save_path
# Define the TensorBoard callback for monitoring training progress
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=10, batch_size=10, write_graph=True)

# Define the ReduceLROnPlateau callback for reducing the learning rate when the model plateaus
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=20, min_lr=10**-20)

# Define the ModelCheckpoint callback for saving the best model based on validation mean absolute error
checkpoint_callback_mae = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_mae',
                                                             mode='min', save_best_only=True)

# Define the ModelCheckpoint callback for saving the best model based on validation loss
checkpoint_callback_loss = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss',
                                                              mode='min', save_best_only=True)

# Define the EarlyStopping callback for stopping training when the model stops improving
early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)

# Define the TerminateOnNaN callback for terminating the training process if there is NaN value in any of the output
terminate_callback = tf.keras.callbacks.TerminateOnNaN()

# Train the model using the defined callbacks
history = seq.fit(x, y, epochs=n_epochs, validation_split=0.1,
                  batch_size=50, callbacks=[early_stop_callback,
                  checkpoint_callback_mae, checkpoint_callback_loss,
                  tb_callback, lr_callback, terminate_callback])
#Save the trained model
seq.save(model_path)

#Save the training history

np.save(f'{save_path}ConvLstm_sst13rf_rf_3SI.npy', history.history)               
              

Adding channel 0 with shape: (2308, 7, 1, 25, 39)
Adding channel 1 with shape: (2308, 7, 1, 25, 39)
INPUT SHAPE -->  (2308, 7, 2, 25, 39)
TARGET SHAPE -->  (2308, 1, 25, 39)


2023-03-20 15:33:36.791759: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 7, 25, 39, 4)      880       
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 7, 25, 39, 8)      3488      
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, 7, 25, 39, 8)      4640      
                                                                 
 conv_lstm2d_3 (ConvLSTM2D)  (None, 7, 25, 39, 16)     13888     
                                                                 
 conv_lstm2d_4 (ConvLSTM2D)  (None, 25, 39, 16)        18496     
                                                                 
 conv2d (Conv2D)             (None, 25, 39, 15)        2175      
                                                                 
 conv2d_1 (Conv2D)           (None, 25, 39, 1)         1

42/42 [==============================] - 273s 6s/step - loss: 202.1010 - mae: 8.3232 - val_loss: 118.1337 - val_mae: 7.1854 - lr: 1.0000e-04
Epoch 45/200
42/42 [==============================] - 265s 6s/step - loss: 201.9785 - mae: 8.3227 - val_loss: 117.9281 - val_mae: 7.1554 - lr: 1.0000e-04
Epoch 46/200
42/42 [==============================] - 256s 6s/step - loss: 201.8569 - mae: 8.3062 - val_loss: 117.8177 - val_mae: 7.1545 - lr: 1.0000e-04
Epoch 47/200
42/42 [==============================] - 265s 6s/step - loss: 201.7466 - mae: 8.3198 - val_loss: 117.6611 - val_mae: 7.1401 - lr: 1.0000e-04
Epoch 48/200
42/42 [==============================] - 262s 6s/step - loss: 201.6030 - mae: 8.2838 - val_loss: 117.5365 - val_mae: 7.1360 - lr: 1.0000e-04
Epoch 49/200
42/42 [==============================] - 254s 6s/step - loss: 201.4902 - mae: 8.2865 - val_loss: 117.4794 - val_mae: 7.1528 - lr: 1.0000e-04
Epoch 50/200
42/42 [==============================] - 269s 6s/step - loss: 201.3414 - mae

42/42 [==============================] - 257s 6s/step - loss: 193.3146 - mae: 7.9578 - val_loss: 109.0704 - val_mae: 6.7181 - lr: 1.0000e-04
Epoch 98/200
42/42 [==============================] - 263s 6s/step - loss: 193.1504 - mae: 7.9076 - val_loss: 109.0110 - val_mae: 6.7493 - lr: 1.0000e-04
Epoch 99/200
42/42 [==============================] - 240s 6s/step - loss: 192.9736 - mae: 7.9308 - val_loss: 108.6258 - val_mae: 6.6759 - lr: 1.0000e-04
Epoch 100/200
42/42 [==============================] - 248s 6s/step - loss: 192.7661 - mae: 7.8921 - val_loss: 108.5800 - val_mae: 6.7152 - lr: 1.0000e-04
Epoch 101/200
42/42 [==============================] - 252s 6s/step - loss: 192.6039 - mae: 7.9151 - val_loss: 108.2993 - val_mae: 6.6764 - lr: 1.0000e-04
Epoch 102/200
42/42 [==============================] - 255s 6s/step - loss: 192.3649 - mae: 7.8816 - val_loss: 108.2668 - val_mae: 6.7149 - lr: 1.0000e-04
Epoch 103/200
42/42 [==============================] - 255s 6s/step - loss: 192.2217 -

Epoch 150/200
42/42 [==============================] - 257s 6s/step - loss: 185.1753 - mae: 7.5236 - val_loss: 101.0004 - val_mae: 6.3236 - lr: 1.0000e-04
Epoch 151/200
42/42 [==============================] - 262s 6s/step - loss: 185.0548 - mae: 7.5192 - val_loss: 100.7784 - val_mae: 6.2866 - lr: 1.0000e-04
Epoch 152/200
42/42 [==============================] - 249s 6s/step - loss: 185.0196 - mae: 7.5202 - val_loss: 100.6496 - val_mae: 6.2729 - lr: 1.0000e-04
Epoch 153/200
42/42 [==============================] - 257s 6s/step - loss: 184.8012 - mae: 7.5173 - val_loss: 100.4687 - val_mae: 6.2421 - lr: 1.0000e-04
Epoch 154/200
42/42 [==============================] - 249s 6s/step - loss: 184.7063 - mae: 7.4931 - val_loss: 100.4439 - val_mae: 6.2586 - lr: 1.0000e-04
Epoch 155/200
42/42 [==============================] - 252s 6s/step - loss: 184.6041 - mae: 7.5172 - val_loss: 100.1453 - val_mae: 6.1788 - lr: 1.0000e-04
Epoch 156/200
42/42 [==============================] - 242s 6s/step - 

In [2]:
def preprocess_data(sequence, n_steps, channels):
    """
    Function to preprocess the data and prepare it for model training.
    
    Parameters:
        sequence (xarray.Dataset): Data sequence to be preprocessed.
        n_steps (int): Number of time steps in the input sequence.
        channels (int): Number of channels in the input sequence.
    
    Returns:
        tuple: A tuple containing the preprocessed input sequence (X) and output sequence (y).
    """
    # Select data for Jun, Jul, Aug, Sept
    sequence = sequence.sel(time=slice("2017-01-01","2018-12-31"))
    sequence = sequence.where(sequence.time.dt.month.isin([6,7,8,9]), drop=True)
    max = sequence.max()
    min = sequence.min()
    
    # Max-min normalization
    sequence = (sequence-min)/(max - min)
    sequence = np.array(sequence)
    
    # Exponential Space Transform
    valid_pts = np.where(sequence==sequence)
    sequence[valid_pts] = np.exp(sequence[valid_pts])
    sequence[valid_pts] = np.power(sequence[valid_pts],7)
    invalid_pts = np.where(sequence != sequence)
    sequence[invalid_pts] = 0
    
    # Prepare input and output sequences
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix + lead_time > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix + lead_time]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


def create_input(channels, n_steps):
    """
    Function to create the input for the model.
    
    Parameters:
        channels (list): List of channels to be used in the input.
        n_steps (int): Number of time steps in the input sequence.
        
    Returns:
        numpy.ndarray: The input for the model.
    """
    stack = []
    for i in range(len(channels)):
        channel_data = xr.open_dataarray(f"/home/cccr/roxy/matin/MTech_project/data/{channels[i]}")
        input, _ = preprocess_data(channel_data, n_steps=n_steps, channels=channels[i])
        input = np.expand_dims(input, axis=2)
        print(f"Adding channel {i} with shape: {input.shape}")
        stack.append(input)
    out = np.dstack(stack)
    return out

channels = ["SIFilteredrfBOB_6lag.nc","SIFilteredSSTBOB_6.nc"]
steps = 7

# Create input for the model
input = create_input(channels, steps)

# Preprocess target data
channel_data = xr.open_dataarray(add_data + "SIFilteredrfBOB_6lag.nc")
_, target = preprocess_data(channel_data, n_steps=steps, channels=channels[-1])
target = np.expand_dims(target, axis=1)

# Roll axes for input and target
x = np.rollaxis(input, 4, 2)
x = np.rollaxis(x, 4, 2)

y = np.rollaxis(target, 3, 1)
y = np.rollaxis(y, 3, 1)

# Assert that input and target shapes are consistent
assert input.shape[0] == target.shape[0]
assert input.shape[-1] == target.shape[-1]
assert input.shape[-2] == target.shape[-2]


# Print shape of input and target
print("INPUT SHAPE --> ", input.shape)
print("TARGET SHAPE --> ", target.shape)

# Delete original input and target variables to free up memory
del input
del target


Adding channel 0 with shape: (234, 7, 1, 25, 39)
Adding channel 1 with shape: (234, 7, 1, 25, 39)
INPUT SHAPE -->  (234, 7, 2, 25, 39)
TARGET SHAPE -->  (234, 1, 25, 39)


In [3]:
model = tf.keras.models.load_model(model_path)
pred = model.predict(x)

actual = y.flatten()
prediction = pred.flatten()
corr = np.corrcoef(actual,prediction)
corr

8/8 [==============================] - 3s 266ms/step


array([[1.        , 0.01787093],
       [0.01787093, 1.        ]])